In [ ]:
 
s = requests.Session()
r = s.get('https://www.espn.com')


SWID = '#######'
espn_s2 = "###########"


league_id = #######


url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %league_id

r = requests.get(url, cookies={"swid": swid, 'espn_s2': espn_s2}).json()

#Get Team IDs
teamId = {}
print(r)
print("\n\n##############\n\n")
for team in r['teams']:
    teamId[team['id']] = team['location'].strip() + ' ' + team['nickname'].strip()


#Get each team's weekly points and calculate their head-to-head records
weeklyPoints = {}
r = requests.get(url, cookies={"swid": swid, 'espn_s2': espn_s2}, params={"view": "mMatchup"}).json()


weeklyPts = pd.DataFrame()
for each in r['schedule']:
    #each = r['schedule'][0]

    week = each['matchupPeriodId']
    if week >= 14:
        continue

    homeTm = teamId[each['home']['teamId']]
    homeTmPts = each['home']['totalPoints']

    try:
        awayTm = teamId[each['away']['teamId']]
        awayTmPts = each['away']['totalPoints']
    except:
        homeTmPts = 'BYE'
        continue

    temp_df = pd.DataFrame(list(zip([homeTm, awayTm], [homeTmPts, awayTmPts], [week, week])), columns=['team','pts','week'])

    if homeTmPts > awayTmPts:
        temp_df.loc[0,'win'] = 1
        temp_df.loc[0,'loss'] = 0
        temp_df.loc[0,'tie'] = 0

        temp_df.loc[1,'win'] = 0
        temp_df.loc[1,'loss'] = 1
        temp_df.loc[1,'tie'] = 0

    elif homeTmPts < awayTmPts:
        temp_df.loc[0,'win'] = 0
        temp_df.loc[0,'loss'] = 1
        temp_df.loc[0,'tie'] = 0

        temp_df.loc[1,'win'] = 1
        temp_df.loc[1,'loss'] = 0
        temp_df.loc[1,'tie'] = 0

    elif homeTmPts == awayTmPts:
        temp_df.loc[0,'win'] = 0
        temp_df.loc[0,'loss'] = 0
        temp_df.loc[0,'tie'] = 1

        temp_df.loc[1,'win'] = 0
        temp_df.loc[1,'loss'] = 0
        temp_df.loc[1,'tie'] = 1

    weeklyPts = weeklyPts.append(temp_df, sort=True).reset_index(drop=True)
print(temp_df)
print(weeklyPts)
weeklyPts['win'] = weeklyPts.groupby(['team'])['win'].cumsum()
weeklyPts['loss'] = weeklyPts.groupby(['team'])['loss'].cumsum()
weeklyPts['tie'] = weeklyPts.groupby(['team'])['tie'].cumsum()



# Calculate each teams record compared to all other teams points week to week
cumWeeklyRecord = {}   
for week in weeklyPts[weeklyPts['pts'] > 0]['week'].unique():
    df = weeklyPts[weeklyPts['week'] == week]

    cumWeeklyRecord[week] = {}
    for idx, row in df.iterrows():
        team = row['team']
        pts = row['pts']
        win = len(df[df['pts'] < pts])
        loss = len(df[df['pts'] > pts])
        tie = len(df[df['pts'] == pts])

        cumWeeklyRecord[week][team] = {}
        cumWeeklyRecord[week][team]['win'] = win
        cumWeeklyRecord[week][team]['loss'] = loss
        cumWeeklyRecord[week][team]['tie'] = tie-1

# Combine those cumluative records to get an overall season record      
overallRecord = {}     
for each in cumWeeklyRecord.items():
    for team in each[1].keys():
        print(each)
        if team not in overallRecord.keys():
            overallRecord[team] = {} 

        win = each[1][team]['win']
        loss = each[1][team]['loss']
        tie = each[1][team]['tie']

        if 'win' not in overallRecord[team].keys():
            overallRecord[team]['win'] = win
        else:
            overallRecord[team]['win'] += win

        if 'loss' not in overallRecord[team].keys():
            overallRecord[team]['loss'] = loss
        else:
            overallRecord[team]['loss'] += loss

        if 'tie' not in overallRecord[team].keys():
            overallRecord[team]['tie'] = tie
        else:
            overallRecord[team]['tie'] += tie


# Little cleaning up of the data nd calculating win %
overallRecord_df = pd.DataFrame(overallRecord).T
overallRecord_df = overallRecord_df.rename_axis('team').reset_index()
overallRecord_df = overallRecord_df.rename(columns={'win':'overall_win', 'loss':'overall_loss','tie':'overall_tie'})
overallRecord_df['overall_win%'] = overallRecord_df['overall_win'] / (overallRecord_df['overall_win'] + overallRecord_df['overall_loss'] + overallRecord_df['overall_tie'])
overallRecord_df['overall_rank'] = overallRecord_df['overall_win%'].rank(ascending=False, method='min')




regularSeasRecord = weeklyPts[weeklyPts['week'] == 13][['team','win','loss', 'tie']]
regularSeasRecord['win%'] = regularSeasRecord['win'] / (regularSeasRecord['win'] + regularSeasRecord['loss'] + regularSeasRecord['tie'])
regularSeasRecord['rank'] = regularSeasRecord['win%'].rank(ascending=False, method='min')



final_df = overallRecord_df.merge(regularSeasRecord, how='left', on=['team'])


In [44]:
import random

league_id = 1822630


url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %league_id

r = requests.get(url).json()


#I forgot to set a seed so the league results may not be the same if you choose to use the randomizer
fake = {'messages': ['You are not authorized to view this League.'], 'details': [{'message': 'You are not authorized to view this League.', 'shortMessage': 'You are not authorized to view this League.', 'resolution': None, 'type': 'AUTH_LEAGUE_NOT_VISIBLE', 'metaData': None}]}
fake2 = {'messages': ['This League has been deleted.'], 'details': [{'message': 'This League has been deleted.', 'shortMessage': 'This League has been deleted.', 'resolution': None, 'type': 'LEAGUE_NOT_FOUND_DELETED', 'metaData': None}]}
fake3 = {'messages': ['Not Found'], 'details': [{'message': 'Not Found', 'shortMessage': 'Not Found', 'resolution': None, 'type': 'GENERAL_NOT_FOUND', 'metaData': None}]}
#the league ids used are in the cell below this, incase you would like to expediate the process
#there are tons of leagues you cannot access or get data from, so this can take a long time to get valid results

results = []
for i in range(50000):
    league =''
    for i in range(len(str(league_id))):
        league += str(random.randint(0,9))
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %league

    r = requests.get(url).json()
    if fake == r or fake2 == r or fake3 == r:
        continue
    results.append(league)
    
    
    

ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')"))

In [5]:
import pandas as pd
import numpy as np
import requests

results = ['4698169', '0652106', '0181007', '0472425', '4474179', '2144520', '3887054', '1567811', '1240334', '0718508', '1263863', '8112249', '2289296', '1270737', '0569204', '7847842', '6823046', '6270327', '8474110', '7246723', '2678957', '1142144', '0358793', '0758174', '5041152', '6906207', '0729553', '1091214', '3146811']
print(len(results))
num_teams = []
twelve_manners = []
for i in results:
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %i

    r = requests.get(url).json()
    
    num_teams.append(len(r['teams']))
    
    if len(r['teams']) == 12:
        twelve_manners.append(i)
    

#twelve_manners.remove('4082961')
#twelve_manners.remove('4293909')
#twelve_manners.remove('5051957')


    
totals = {}
for i in num_teams:
    if i in totals.keys():
        totals[i] += 1
    else:
        totals[i] = 1
    
print(totals)
final_dfs = []
for i in range(len(twelve_manners)):
    print('Working on league id', twelve_manners[i])
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %twelve_manners[i]

    r = requests.get(url).json()

    #Get Team IDs
    teamId = {}
    #print(r)
    #print("\n\n##############\n\n")
    for team in r['teams']:
        teamId[team['id']] = team['location'].strip() + ' ' + team['nickname'].strip()


    #Get each team's weekly points and calculate their head-to-head records
    weeklyPoints = {}
    r = requests.get(url, params={"view": "mMatchup"}).json()


    weeklyPts = pd.DataFrame()
    for each in r['schedule']:
        #each = r['schedule'][0]

        week = each['matchupPeriodId']
        if week >= 14:
            continue

        homeTm = teamId[each['home']['teamId']]
        homeTmPts = each['home']['totalPoints']

        try:
            awayTm = teamId[each['away']['teamId']]
            awayTmPts = each['away']['totalPoints']
        except:
            homeTmPts = 'BYE'
            continue

        temp_df = pd.DataFrame(list(zip([homeTm, awayTm], [homeTmPts, awayTmPts], [week, week])), columns=['team','pts','week'])

        if homeTmPts > awayTmPts:
            temp_df.loc[0,'win'] = 1
            temp_df.loc[0,'loss'] = 0
            temp_df.loc[0,'tie'] = 0

            temp_df.loc[1,'win'] = 0
            temp_df.loc[1,'loss'] = 1
            temp_df.loc[1,'tie'] = 0

        elif homeTmPts < awayTmPts:
            temp_df.loc[0,'win'] = 0
            temp_df.loc[0,'loss'] = 1
            temp_df.loc[0,'tie'] = 0

            temp_df.loc[1,'win'] = 1
            temp_df.loc[1,'loss'] = 0
            temp_df.loc[1,'tie'] = 0

        elif homeTmPts == awayTmPts:
            temp_df.loc[0,'win'] = 0
            temp_df.loc[0,'loss'] = 0
            temp_df.loc[0,'tie'] = 1

            temp_df.loc[1,'win'] = 0
            temp_df.loc[1,'loss'] = 0
            temp_df.loc[1,'tie'] = 1

        weeklyPts = weeklyPts.append(temp_df, sort=True).reset_index(drop=True)
    #print(temp_df)
    #print(weeklyPts)
    weeklyPts['win'] = weeklyPts.groupby(['team'])['win'].cumsum()
    weeklyPts['loss'] = weeklyPts.groupby(['team'])['loss'].cumsum()
    weeklyPts['tie'] = weeklyPts.groupby(['team'])['tie'].cumsum()



    # Calculate each teams record compared to all other teams points week to week
    cumWeeklyRecord = {}   
    for week in weeklyPts[weeklyPts['pts'] > 0]['week'].unique():
        df = weeklyPts[weeklyPts['week'] == week]

        cumWeeklyRecord[week] = {}
        for idx, row in df.iterrows():
            team = row['team']
            pts = row['pts']
            win = len(df[df['pts'] < pts])
            loss = len(df[df['pts'] > pts])
            tie = len(df[df['pts'] == pts])

            cumWeeklyRecord[week][team] = {}
            cumWeeklyRecord[week][team]['win'] = win
            cumWeeklyRecord[week][team]['loss'] = loss
            cumWeeklyRecord[week][team]['tie'] = tie-1

    # Combine those cumluative records to get an overall season record      
    overallRecord = {}     
    for each in cumWeeklyRecord.items():
        for team in each[1].keys():
            #print(each)
            if team not in overallRecord.keys():
                overallRecord[team] = {} 

            win = each[1][team]['win']
            loss = each[1][team]['loss']
            tie = each[1][team]['tie']

            if 'win' not in overallRecord[team].keys():
                overallRecord[team]['win'] = win
            else:
                overallRecord[team]['win'] += win

            if 'loss' not in overallRecord[team].keys():
                overallRecord[team]['loss'] = loss
            else:
                overallRecord[team]['loss'] += loss

            if 'tie' not in overallRecord[team].keys():
                overallRecord[team]['tie'] = tie
            else:
                overallRecord[team]['tie'] += tie


    # Little cleaning up of the data nd calculating win %
    overallRecord_df = pd.DataFrame(overallRecord).T
    overallRecord_df = overallRecord_df.rename_axis('team').reset_index()
    overallRecord_df = overallRecord_df.rename(columns={'win':'overall_win', 'loss':'overall_loss','tie':'overall_tie'})
    overallRecord_df['overall_win%'] = overallRecord_df['overall_win'] / (overallRecord_df['overall_win'] + overallRecord_df['overall_loss'] + overallRecord_df['overall_tie'])
    overallRecord_df['overall_rank'] = overallRecord_df['overall_win%'].rank(ascending=False, method='first')


    regularSeasRecord = weeklyPts[weeklyPts['week'] == 13][['team','win','loss', 'tie']]
    regularSeasRecord['win%'] = regularSeasRecord['win'] / (regularSeasRecord['win'] + regularSeasRecord['loss'] + regularSeasRecord['tie'])
    regularSeasRecord['rank'] = regularSeasRecord['win%'].rank(ascending=False, method='first')


    final_df = overallRecord_df.merge(regularSeasRecord, how='left', on=['team'])
    
    final_dfs.append(final_df)
    
    
    

29
{12: 29}
Working on league id 4698169
Working on league id 0652106
Working on league id 0181007
Working on league id 0472425
Working on league id 4474179
Working on league id 2144520
Working on league id 3887054
Working on league id 1567811
Working on league id 1240334
Working on league id 0718508
Working on league id 1263863
Working on league id 8112249
Working on league id 2289296
Working on league id 1270737
Working on league id 0569204
Working on league id 7847842
Working on league id 6823046
Working on league id 6270327
Working on league id 8474110
Working on league id 7246723
Working on league id 2678957
Working on league id 1142144
Working on league id 0358793
Working on league id 0758174
Working on league id 5041152
Working on league id 6906207
Working on league id 0729553
Working on league id 1091214
Working on league id 3146811


In [120]:
len(final_final_dfs)

28

In [6]:

SAVE_THIS = ['4698169', '0652106', '0181007', '0472425', '4474179', '2144520', '3887054', '1567811', '1240334', '0718508', '1263863', '8112249', '2289296', '1270737', '0569204', '7847842', '6823046', '6270327', '8474110', '7246723', '2678957', '1142144', '0358793', '0758174', '5041152', '6906207', '0729553', '1091214', '3146811']

final_final_dfs = []
for j in range(len(final_dfs)):
    temp = final_dfs[j].iloc[0:12,6:7]
    for i in range(12):
        if pd.isnull(temp.iloc[i,0]):
            break
        if i == 11:
            tempy = final_dfs[j]
            tempy["overall_rank"] = tempy['win'].rank(ascending=False, method='first')
            final_final_dfs.append(final_dfs[j])



playoff_perc = {}
for i in range(14):
    playoff_perc[i] = [0,0]
    
print(final_dfs[1].iloc[0:12,5:7].iloc[0,0])


for j in range(len(final_final_dfs)):
    temp = final_final_dfs[j].iloc[0:12,5:7]
    for i in range(12):
        playoff_perc[temp.iloc[i,1]][1] += 1
        if int(temp.iloc[i,0]) <= 6:
            playoff_perc[temp.iloc[i,1]][0]  += 1

        
print(playoff_perc)
#for league in final_dfs:
#    league


bye_perc = {}
for i in range(14):
    bye_perc[i] = [0,0]
    
print(final_dfs[1].iloc[0:12,5:7].iloc[0,0])


for j in range(len(final_final_dfs)):
    temp = final_final_dfs[j].iloc[0:12,5:7]
    for i in range(12):
        bye_perc[temp.iloc[i,1]][1] += 1
        if int(temp.iloc[i,0]) <= 2:
            bye_perc[temp.iloc[i,1]][0]  += 1
            if temp.iloc[i,1] < 8:
                print(temp)



print(bye_perc)
print(temp.iloc[i,1])



3.0
{0: [0, 0], 1: [0, 5], 2: [0, 11], 3: [0, 21], 4: [0, 32], 5: [0, 41], 6: [5, 47], 7: [47, 63], 8: [50, 50], 9: [39, 39], 10: [15, 15], 11: [10, 10], 12: [2, 2], 13: [0, 0]}
3.0
{0: [0, 0], 1: [0, 5], 2: [0, 11], 3: [0, 21], 4: [0, 32], 5: [0, 41], 6: [0, 47], 7: [0, 63], 8: [8, 50], 9: [21, 39], 10: [15, 15], 11: [10, 10], 12: [2, 2], 13: [0, 0]}
5.0


In [8]:
import plotly.graph_objects as go


playoffs = {0: [0, 0], 1: [0, 5], 2: [0, 11], 3: [0, 21], 4: [0, 32], 5: [0, 41], 6: [5, 47], 7: [47, 63], 8: [50, 50], 9: [39, 39], 10: [15, 15], 11: [10, 10], 12: [2, 2], 13: [0, 0]}
byes = {0: [0, 0], 1: [0, 5], 2: [0, 11], 3: [0, 21], 4: [0, 32], 5: [0, 41], 6: [0, 47], 7: [0, 63], 8: [8, 50], 9: [21, 39], 10: [15, 15], 11: [10, 10], 12: [2, 2], 13: [0, 0]}

winny = []
for i in range(0,14):
    winny.append(i)
bye_percs = []
playoff_percs = []
for i in range(0,14):
    if i == 0:
        playoff_percs.append(0)
        bye_percs.append(0)
    elif i == 13:
        playoff_percs.append(100)
        bye_percs.append(100)
    else:
        playoff_percs.append(round(playoffs[i][0]/playoffs[i][1]*100))
        bye_percs.append(round(byes[i][0]/byes[i][1]*100))



fig = go.Figure()
fig.add_trace(go.Bar(
    x=winny,
    y=playoff_percs,
    name='Make the Playoffs',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=winny,
    y=bye_percs,
    name='Have a Bye',
    marker_color='lightsalmon'
))


fig.update_layout(
    title="Wins Needed to Get Byes or Make Playoffs",
    xaxis_title="Wins",
    yaxis_title="Percent of Outcomes",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [79]:
temp = [0]
for item in Counts:
    temp.append(item)
predicted_bye_perc = 0
for i in range(len(temp)):
    predicted_bye_perc = predicted_bye_perc + (temp[i]/100)*(bye_percs[i]/100)

temp = [0]
for item in Counts:
    temp.append(item)
predicted_playoff_perc = 0
for i in range(len(temp)):
    predicted_playoff_perc = predicted_playoff_perc + (temp[i]/100)*(playoff_percs[i]/100)

print('Percent of countcomes predicted to make playoffs: ' + str(round(predicted_playoff_perc*10000)/100)+'%')
print('Percent of countcomes predicted to get playoff bye: ' + str(round(predicted_bye_perc*10000)/100) +'%')


['4698169', '0652106', '0181007', '0472425', '4082961', '4474179', '2144520', '3887054', '1567811', '1240334', '0718508', '1263863', '8112249', '2289296', '4293909', '1270737', '0569204', '7847842', '6823046', '6270327', '8474110', '7246723', '2678957', '5051957', '1142144', '0358793', '0758174', '5041152', '6906207', '0729553', '1091214', '3146811']


ValueError: list.remove(x): x not in list

In [47]:
import matplotlib.pyplot as plt
totals = {}
for i in num_teams:
    if i in totals.keys():
        totals[i] += 1
    else:
        totals[i] = 1
    
print(totals)








{10: 75, 12: 32, 8: 28, 6: 5, 14: 4, 4: 2, 16: 2, 18: 1}


In [17]:
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %results[0]

r = requests.get(url).json()
fake2 = r
print(r)
print(fake)

{'messages': ['This League has been deleted.'], 'details': [{'message': 'This League has been deleted.', 'shortMessage': 'This League has been deleted.', 'resolution': None, 'type': 'LEAGUE_NOT_FOUND_DELETED', 'metaData': None}]}
{'messages': ['You are not authorized to view this League.'], 'details': [{'message': 'You are not authorized to view this League.', 'shortMessage': 'You are not authorized to view this League.', 'resolution': None, 'type': 'AUTH_LEAGUE_NOT_VISIBLE', 'metaData': None}]}


In [5]:
import requests

league_id = 0


url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/%s' %league_id

r = requests.get(url).json()

print(type(r))
print(r[])

<class 'dict'>
{'messages': ['Not Found'], 'details': [{'message': 'Not Found', 'shortMessage': 'Not Found', 'resolution': None, 'type': 'GENERAL_NOT_FOUND', 'metaData': None}]}
